In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Función para cargar y mostrar imágenes, tomando como base el paso de "Comprobación rápida"

In [ ]:
def load_image(img_path):
    """
    Carga una imagen desde la ruta dada (BGR) y la convierte a RGB para visualizar con Matplotlib.
    Retorna la imagen en formato RGB y, además, la original en escala de grises.
    """
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        raise ValueError(f"No se pudo cargar la imagen en la ruta: {img_path}")

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    return img_rgb, img_gray
    

def show_image(img_rgb, title="Imagen"):
    """
    Muestra una imagen en RGB con Matplotlib.
    """
    plt.imshow(img_rgb)
    plt.title(title)
    plt.axis("off")
    plt.show()

# Implementación del Filtro de Paso Alto Gaussiano (basado en el método descrito en Saha (2014) para obtener contenido de alta frecuencia)

In [ ]:
def high_pass_filter_gaussian(img_gray, ksize=5):
    """
    Aplica un filtro de paso alto a la imagen en escala de grises (img_gray),
    restando la versión suavizada (Gaussiana) de la imagen original.
    Retorna la imagen realzada en altas frecuencias.
    
    ksize: Tamaño del kernel del filtro Gaussiano (debe ser impar).
    """
    # Filtro Gaussiano (paso bajo)
    blurred = cv2.GaussianBlur(img_gray, (ksize, ksize), 0)
    # Paso alto = imagen original - imagen suavizada
    high_freq = cv2.subtract(img_gray, blurred)
    return high_freq

# Implementación del Filtro de Paso Alto con Filtro de la Media (visto en las transparencias de la asignatura)

In [ ]:
def high_pass_filter_mean(img_gray, ksize=5):
    """
    Aplica un filtro de paso alto a la imagen en escala de grises (img_gray),
    restando la versión suavizada (Media) de la imagen original.
    Retorna la imagen realzada en altas frecuencias.
    
    ksize: Tamaño del kernel del filtro media (debe ser impar).
    """
    # Filtro Media (paso bajo)
    blurred = cv2.blur(img_gray, (ksize, ksize))
    # Paso alto = imagen original - imagen suavizada
    high_freq = cv2.subtract(img_gray, blurred)
    return high_freq

# Implementación del Filtro de Paso Alto con Filtro de la Mediana (visto en las transparencias de la asignatura)

In [ ]:
def high_pass_filter_median(img_gray, ksize=5):
    """
    Aplica un filtro de paso alto a la imagen en escala de grises (img_gray),
    restando la versión suavizada (Mediana) de la imagen original.
    Retorna la imagen realzada en altas frecuencias.
    
    ksize: Tamaño del kernel del filtro mediana (debe ser impar).
    """
    # Filtro Mediana (paso bajo)
    blurred = cv2.medianBlur(img_gray, ksize)
    # Paso alto = imagen original - imagen suavizada
    high_freq = cv2.subtract(img_gray, blurred)
    return high_freq

# Métrica básica para cuantificar nitidez usando el contenido de alta frecuencia

In [ ]:
def metric_highfreq(img_gray, ksize=5, filter_function=high_pass_filter_gaussian):
    """
    Calcula una medida de nitidez basada en la 'energía' (o varianza) 
    de la imagen de alta frecuencia.
    """
    hf = filter_function(img_gray, ksize=ksize)
    # Por ejemplo, podríamos usar la varianza como aproximación
    return np.var(hf)

In [ ]:
def process_all_images_in_folders(
    input_root="images",
    output_root="images_high_frec",
    ksize=5,
    filter="gaussian"
):
    """
    Recorre todas las subcarpetas dentro de 'input_root'.
    - Para cada archivo .png, calcula el contenido de alta frecuencia.
    - Guarda la imagen en 'output_root' manteniendo la misma estructura de subcarpetas.
    - Genera un archivo .txt con la varianza de cada imagen procesada.
    """
    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(output_root, exist_ok=True)

    # Definir la función de filtro a utilizar
    if filter.lower() == "gaussian":
        filter_function = high_pass_filter_gaussian
    elif filter.lower() == "mean":
        filter_function = high_pass_filter_mean
    elif filter.lower() == "median":
        filter_function = high_pass_filter_median
    else:
        raise ValueError(f"El filtro '{filter}' no está soportado.")

    # Iterar sobre todas las subcarpetas de 'input_root'
    for subfolder in sorted(os.listdir(input_root)):
        subfolder_path = os.path.join(input_root, subfolder)
        
        # Verificamos si efectivamente es una carpeta
        if not os.path.isdir(subfolder_path):
            continue
        
        # Crear subcarpeta de salida correspondiente en 'output_root'
        output_subfolder = os.path.join(output_root, subfolder)
        output_subfolder = os.path.join(output_subfolder, filter)
        os.makedirs(output_subfolder, exist_ok=True)
        
        # Crear/abrir el archivo txt donde guardaremos la info de varianza
        txt_path = os.path.join(output_subfolder, "info_varianzas" + "_" + filter + ".txt")
        
        with open(txt_path, "w", encoding="utf-8") as f_txt:
            # Escribir descripción breve de qué significan esos valores
            f_txt.write("Este archivo contiene la varianza de alta frecuencia para cada imagen de la carpeta:\n")
            f_txt.write(f"{subfolder_path}\n\n")
            f_txt.write("La varianza indica la dispersión de los valores de alta frecuencia.\n")
            f_txt.write("Valores mayores suelen corresponder a más bordes (y a veces mayor nitidez).\n\n")
            
            # Recorremos los archivos dentro de la subcarpeta
            for filename in sorted(os.listdir(subfolder_path)):
                if filename.lower().endswith(".png"):
                    # Ruta completa del archivo de entrada
                    input_image_path = os.path.join(subfolder_path, filename)
                    
                    # Leer la imagen
                    img_bgr = cv2.imread(input_image_path)
                    if img_bgr is None:
                        continue
                    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

                    # Obtener contenido de alta frecuencia
                    hf_img = filter_function(img_gray, ksize=ksize)
                    
                    # Calcular la varianza
                    var_val = metric_highfreq(img_gray, ksize=ksize, filter_function=filter_function)
                    
                    # Para guardar la imagen con un contraste parecido al autoescalado de matplotlib:
                    hf_min, hf_max = hf_img.min(), hf_img.max()
                    if hf_max > hf_min:
                        hf_scaled = (hf_img - hf_min) / (hf_max - hf_min) * 255.0
                    else:
                        # Si la imagen es prácticamente uniforme, no se puede escalar
                        hf_scaled = hf_img.astype(np.float32)
                    
                    hf_scaled = hf_scaled.astype(np.uint8)
                    
                    # Construir el nombre de salida
                    name, ext = os.path.splitext(filename)
                    output_image_name = f"{name}_high_frec" + "_" + filter + ".png"
                    output_image_path = os.path.join(output_subfolder, output_image_name)
                    
                    # Guardar la imagen de alta frecuencia
                    cv2.imwrite(output_image_path, hf_scaled)
                    
                    # Escribir en el txt la varianza obtenida
                    line = f"{filename} -> varianza alta frecuencia: {var_val:.3f}\n"
                    f_txt.write(line)

In [ ]:
process_all_images_in_folders(
    input_root="images",
    output_root="images_high_frec",
    ksize=5,
    filter="mean"
)
print("Procesamiento completado.")